In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv",index_col='PassengerId')
train.head()

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv",index_col='PassengerId')

In [ ]:
train.info()

# Prepare Pipeline

In [ ]:
y = train.pop('Survived')
y.isnull().sum()

In [ ]:
train.info()

In [ ]:
num_col = [col for col in train.columns if train[col].dtype in ['int64','float64']]
num_col

In [ ]:
cat_cols = [col for col in train.columns if train[col].dtype == 'O']
cat_cols

In [ ]:
card = [train[col].nunique() for col in cat_cols]
card

In [ ]:
two_cat = ['Sex','Embarked']

In [ ]:
train.hist()

In [ ]:
X_train_sel = train[num_col+two_cat]
X_train_sel.head()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
my_num_preproc = Pipeline(steps=[
    ('imputer',SimpleImputer()),
    ('scaler',StandardScaler())
])

In [ ]:
my_cat_preproc = Pipeline(steps=[
    ('cat_imputer',SimpleImputer(strategy='most_frequent')),
    ('ord',OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
my_tranf = ColumnTransformer([
    ('num',my_num_preproc,num_col),
    ('cat',my_cat_preproc,two_cat)
])

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# forest = RandomForestClassifier()

In [ ]:
# from xgboost import XGBClassifier
# xgboost =  XGBClassifier(alpha=14,
#                      eta= 0.1,
#                      max_depth=6,
#                         reg_lambda=5)

In [ ]:
from xgboost import XGBClassifier
xgboost =  XGBClassifier()

In [ ]:
my_pipe = Pipeline(steps=[
    ('preproc',my_tranf),
    ('model',xgboost)])

In [ ]:
from sklearn.model_selection import GridSearchCV 

In [ ]:
params = [
    {'model__eta':[0.05,0.1],
     'model__max_depth':[6],
     'model__lambda':[15,5],
     'model__alpha':[21,14]
    }
]

In [ ]:
# params = [
#     {'model__n_estimators':[10,100],
#      'model__min_samples_split':[2,3,4],
#      'model__min_impurity_decrease':[0,1,10],
#      'model__bootstrap':[True,False]
#     }
# ]

In [ ]:
gs = GridSearchCV(my_pipe,params,cv=5,scoring='accuracy',n_jobs=-1,verbose=1)

In [ ]:
# gs = my_pipe

In [ ]:
gs.fit(X_train_sel,y)

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
pred = pd.DataFrame(gs.predict(test[num_col+two_cat]))
pred.index = test[num_col+two_cat].index
pred

In [ ]:
pred.columns = ['Survived']

In [ ]:
pred

In [ ]:
pred.to_csv('out.txt')